## Run the following cells after cloning onto a Jupyter Chameleon (Clone repo into /tmp/deeptrust)

In [ ]:
cd /work/deeptrust/src/continuous-x/
chmod 777 *.sh


## ADD LEASES

In [ ]:
sed -i 's/'"<REDACTED_RESERVATION_LEASE1>"'/'"<LEASE>"'/g' /work/deeptrust/src/continuous-x/tf/tacc/main.tf
sed -i 's/'"<REDACTED_RESERVATION_LEASE2>"'/'"<LEASE>"'/g' /work/deeptrust/src/continuous-x/tf/tacc/main.tf


In [ ]:
./download-libraries.sh

In [ ]:
export PATH=/work/.local/bin:$PATH
export PYTHONUSERBASE=/work/.local
PYTHONUSERBASE=/work/.local pip install --user -r /work/deeptrust/src/continuous-x/ansible/k8s/kubespray/requirements.txt


## Provision resources using terraform - Provisions KVM/Baremetals/Network/Block Storage/Object Storage

In [ ]:
./provision-terraform.sh

## Configuring kubernetes on the host

In [ ]:
export PATH=/work/.local/bin:$PATH
export PYTHONUSERBASE=/work/.local
export ANSIBLE_CONFIG=/work/deeptrust/src/continuous-x/ansible/ansible.cfg
export ANSIBLE_ROLES_PATH=roles
cd /work/deeptrust/src/continuous-x/ansible
ansible-playbook -i inventory.yml pre_k8s/pre_k8s_configure.yml


## Load Data into Object Storage

In [ ]:
export ANSIBLE_CONFIG=/work/deeptrust/src/continuous-x/ansible/ansible.cfg
export ANSIBLE_ROLES_PATH=roles
ansible-playbook -i inventory.yml ../ansible-baremetal/setup/bootstrap.yml


In [ ]:
export ANSIBLE_CONFIG=/work/deeptrust/src/continuous-x/ansible/ansible.cfg
export ANSIBLE_ROLES_PATH=roles
ansible-playbook -i inventory.yml pre_k8s/set_up_object_storage.yml

In [ ]:
export ANSIBLE_CONFIG=/work/deeptrust/src/continuous-x/ansible/ansible.cfg
export ANSIBLE_ROLES_PATH=roles
cd /work/deeptrust/src/continuous-x/ansible/k8s/kubespray
ansible-playbook -i ../inventory/mycluster --become --become-user=root ./cluster.yml


In [ ]:
export ANSIBLE_CONFIG=/work/deeptrust/src/continuous-x/ansible/ansible.cfg
export ANSIBLE_ROLES_PATH=roles
cd /work/deeptrust/src/continuous-x/ansible
ansible-playbook -i inventory.yml post_k8s/post_k8s_configure.yml

## Configure Block Storage - Partitioning

In [ ]:
#Set up Block Storage
export ANSIBLE_CONFIG=/work/deeptrust/src/continuous-x/ansible/ansible.cfg
export ANSIBLE_ROLES_PATH=roles
cd /work/deeptrust/src/continuous-x/ansible
ansible-playbook -i inventory.yml post_k8s/set_up_block_storage.yml

## Create Images for Prometheus and Grafana that have all the relevant configurations baked into the image

In [ ]:
export ANSIBLE_CONFIG=/work/deeptrust/src/continuous-x/ansible/ansible.cfg
export ANSIBLE_ROLES_PATH=roles
cd /work/deeptrust/src/continuous-x/ansible
ansible-playbook -i inventory.yml ../argocd/initial_container_create.yml

## Creates Platform with Minio, MLFlow, LabelStudio, Grafana, Prometheus, Postgres

In [ ]:
cd /work/deeptrust/src/continuous-x/ansible
ansible-playbook -i inventory.yml ../argocd/argocd_add_platform.yml

Enable port forwarding for services <br>
ssh -L 8443:127.0.0.1:8443 -i ~/.ssh/id_rsa_chameleon cc@A.B.C.D <br>
kubectl port-forward -n kube-system svc/kubernetes-dashboard 8443:443 <br>
https://127.0.0.1:8443/ <br>

ssh -L 8888:127.0.0.1:8888 -i ~/.ssh/id_rsa_chameleon cc@A.B.C.D <br>
kubectl port-forward svc/argocd-server -n argocd 8888:443 <br>
https://127.0.0.1:8888/ <br>

ssh -L 2746:127.0.0.1:2746 -i ~/.ssh/id_rsa_chameleon cc@A.B.C.D <br>
kubectl -n argo port-forward svc/argo-server 2746:2746 <br>
https://127.0.0.1:2746/ <br>

## BareMetal Setup - Clones repo to run required docker files, Mounts Object Storage, Starts containers on baremetals

In [ ]:
#Set up baremetals
export ANSIBLE_CONFIG=/work/deeptrust/src/continuous-x/ansible-baremetal/ansible.cfg
export ANSIBLE_ROLES_PATH=roles
cd /work/deeptrust/src/continuous-x/ansible-baremetal
ansible-playbook -i inventory.yml setup/bootstrap.yml

In [ ]:
#Mount Object Storage (Can only be run after initial github repo is fetched in bootstrap.yml)
export ANSIBLE_CONFIG=/work/deeptrust/src/continuous-x/ansible-baremetal/ansible.cfg
export ANSIBLE_ROLES_PATH=roles
cd /work/deeptrust/src/continuous-x/ansible-baremetal
ansible-playbook -i inventory.yml setup/mount_object_storage.yml

In [ ]:
export ANSIBLE_CONFIG=/work/deeptrust/src/continuous-x/ansible-baremetal/ansible.cfg
export ANSIBLE_ROLES_PATH=roles
cd /work/deeptrust/src/continuous-x/ansible-baremetal
ansible-playbook -i inventory.yml setup/containers_bootstrap.yml

## Add Staging, Canary, Production Environments

In [ ]:
export ANSIBLE_CONFIG=/work/deeptrust/src/continuous-x/ansible/ansible.cfg
export ANSIBLE_ROLES_PATH=roles
cd /work/deeptrust/src/continuous-x/ansible
ansible-playbook -i inventory.yml ../argocd/workflow_build_init.yml

In [ ]:

cd /work/deeptrust/src/continuous-x/ansible
ansible-playbook -i inventory.yml ../argocd/argocd_add_staging.yml

In [ ]:

cd /work/deeptrust/src/continuous-x/ansible
ansible-playbook -i inventory.yml ../argocd/argocd_add_canary.yml

In [ ]:

cd /work/deeptrust/src/continuous-x/ansible
ansible-playbook -i inventory.yml ../argocd/argocd_add_production.yml

## Upload workflow templates on ArgoCD

In [ ]:
cd /work/deeptrust/src/continuous-x/ansible
ansible-playbook -i inventory.yml ../argocd/workflow-templates-apply.yml

## Online Data Testing

In [ ]:
export ANSIBLE_CONFIG=/work/deeptrust/src/continuous-x/ansible/ansible.cfg
export ANSIBLE_ROLES_PATH=roles
cd /work/deeptrust/src/continuous-x/ansible-baremetal
ansible-playbook -i inventory.yml ../ansible-baremetal/testing/online-data-testing.yml